## Final Project
### Data Science LHL Project: Restaurant Reccomenation system

### Part 3: Building Model

By: Chloe Phuong

In [971]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import string
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import PorterStemmer
import re
from nltk.stem import WordNetLemmatizer
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
import difflib
from geopy.distance import geodesic
from gensim.models import Word2Vec
import scipy
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [972]:
final_restrev_df = pd.read_json('final_restrev_df')

# Display the DataFrame
final_restrev_df

,business_id,rest_name,address,city,postal_code,latitude,longitude,rest_avg_stars,total_reviews,is_open,categories,review_id,user_id,review_stars,review_text
0,Ohengzi4viLEXqhK50aLxg,Jaclyn's,"1588 Queen Street E, Toronto, ON M4L 1G1, Canada",Toronto,M4L 1G1,43.66627,-79.31802,4.5,15,True,Caribbean,sR9e_G0JdycDTGV4S-Ma2w,qXztukjlJFpu3T-EmSWIIQ,5,"Incredible, the chicken is so tender, seasonin..."
1,Ohengzi4viLEXqhK50aLxg,Jaclyn's,"1588 Queen Street E, Toronto, ON M4L 1G1, Canada",Toronto,M4L 1G1,43.66627,-79.31802,4.5,15,True,Caribbean,ArpqFM_KEDrP0fJScO09Zw,KhpA23RweKsO9TDaGpE2xg,4,We were able to check this place out yesterday...
2,Ohengzi4viLEXqhK50aLxg,Jaclyn's,"1588 Queen Street E, Toronto, ON M4L 1G1, Canada",Toronto,M4L 1G1,43.66627,-79.31802,4.5,15,True,Caribbean,w56U5WwwCzd2Sz9aXsuXjQ,XbiKsujS_qxU3xsr0xUqmQ,4,I think this will be our go-to before any even...
3,Ohengzi4viLEXqhK50aLxg,Jaclyn's,"1588 Queen Street E, Toronto, ON M4L 1G1, Canada",Toronto,M4L 1G1,43.66627,-79.31802,4.5,15,True,Caribbean,sR9e_G0JdycDTGV4S-Ma2w,qXztukjlJFpu3T-EmSWIIQ,5,"Incredible, the chicken is so tender, seasonin..."
4,Ohengzi4viLEXqhK50aLxg,Jaclyn's,"1588 Queen Street E, Toronto, ON M4L 1G1, Canada",Toronto,M4L 1G1,43.66627,-79.31802,4.5,15,True,Caribbean,ArpqFM_KEDrP0fJScO09Zw,KhpA23RweKsO9TDaGpE2xg,4,We were able to check this place out yesterday...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5575,HHvO4MgBFabsmlInou6viQ,Le Baratin,"1600 Dundas Street. W, Toronto, ON M6K 1T8, Ca...",Toronto,M6K 1T8,43.65007,-79.43510,4.5,130,True,French,ZcWCQcO_eKKZ0Z5mdY6K-g,ykWqtFWldi0bojZ6rorFgg,1,This is one of our favourite go to-s in the ne...
5576,HHvO4MgBFabsmlInou6viQ,Le Baratin,"1600 Dundas Street. W, Toronto, ON M6K 1T8, Ca...",Toronto,M6K 1T8,43.65007,-79.43510,4.5,130,True,French,xTaL-8592nVx7mjYKz1f8g,C282JQOYkRWeAjAcfKlK1g,4,Great spot in Parkdale serving French course m...
5577,HHvO4MgBFabsmlInou6viQ,Le Baratin,"1600 Dundas Street. W, Toronto, ON M6K 1T8, Ca...",Toronto,M6K 1T8,43.65007,-79.43510,4.5,130,True,French,-bZ8wE7TaRDkMuBeLM91zQ,RD365MaKbCCjpZQ2KkxROg,5,This felt just like a typical Parisian restaur...
5578,HHvO4MgBFabsmlInou6viQ,Le Baratin,"1600 Dundas Street. W, Toronto, ON M6K 1T8, Ca...",Toronto,M6K 1T8,43.65007,-79.43510,4.5,130,True,French,ZcWCQcO_eKKZ0Z5mdY6K-g,ykWqtFWldi0bojZ6rorFgg,1,This is one of our favourite go to-s in the ne...


In [973]:
final_restrev_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5580 entries, 0 to 5579
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   business_id     5580 non-null   object 
 1   rest_name       5580 non-null   object 
 2   address         5580 non-null   object 
 3   city            5580 non-null   object 
 4   postal_code     5580 non-null   object 
 5   latitude        5580 non-null   float64
 6   longitude       5580 non-null   float64
 7   rest_avg_stars  5580 non-null   float64
 8   total_reviews   5580 non-null   int64  
 9   is_open         5580 non-null   bool   
 10  categories      5580 non-null   object 
 11  review_id       5580 non-null   object 
 12  user_id         5580 non-null   object 
 13  review_stars    5580 non-null   int64  
 14  review_text     5580 non-null   object 
dtypes: bool(1), float64(3), int64(2), object(9)
memory usage: 659.4+ KB


In [974]:
# Lowercasing
final_restrev_df['categories'] = final_restrev_df['categories'].str.lower()

# Removal of Punctuation
import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    '''Custom function to remove punctuation'''
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

final_restrev_df['categories'] = final_restrev_df['categories'].apply(lambda text: remove_punctuation(text))

# Removal of Stopwords
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    '''Custom function to remove stopwords'''
    return ' '.join([word for word in str(text).split() if word not in STOPWORDS])

final_restrev_df['categories'] = final_restrev_df['categories'].apply(lambda text: remove_stopwords(text))

In [975]:
# Lower Casing
final_restrev_df['review_text'] = final_restrev_df['review_text'].str.lower()

## Removal of Puctuations
import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    '''custom function to remove the punctuation'''
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

final_restrev_df['review_text'] = final_restrev_df['review_text'].apply(lambda text: remove_punctuation(text))

## Removal of Stopwords
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    '''custom function to remove the stopwords'''
    return ' '.join([word for word in str(text).split() if word not in STOPWORDS])

final_restrev_df['review_text'] = final_restrev_df['review_text'].apply(lambda text: remove_stopwords(text))

In [976]:
# RESTAURANT NAMES:
restaurant_names = list(final_restrev_df['rest_name'].unique())
def get_top_words(column, top_nu_of_words, nu_of_word):
    vec = CountVectorizer(ngram_range= nu_of_word, stop_words='english')
    bag_of_words = vec.fit_transform(column)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:top_nu_of_words]

In [1010]:
def restaurant_recommendations(rest_name, rating_tolerance):
    # Filter restaurants based on rating tolerance
    filtered_restaurants = final_restrev_df[final_restrev_df['rest_avg_stars'] >= rating_tolerance]

    # Drop duplicates based on rest_name
    filtered_restaurants = filtered_restaurants.drop_duplicates(subset='rest_name', keep='first').reset_index(drop=True)

    # Find the restaurant with the provided rest_name
    target_restaurant = filtered_restaurants[filtered_restaurants['rest_name'] == rest_name]

    if target_restaurant.empty:
        return "The provided restaurant name is not found in the dataset."

    # Calculate the similarity scores based on categories
    category_vectorizer = TfidfVectorizer()
    category_matrix = category_vectorizer.fit_transform(filtered_restaurants['categories'])
    category_similarity = cosine_similarity(category_matrix)

    # Calculate the similarity scores based on review text
    review_vectorizer = TfidfVectorizer()
    review_matrix = review_vectorizer.fit_transform(filtered_restaurants['review_text'])
    review_similarity = cosine_similarity(review_matrix)

    # Combine the similarity scores with a weight of 7:3 (categories:review_text)
    combined_similarity = 0.7 * category_similarity + 0.3 * review_similarity

    # Get the index of the target restaurant within the filtered restaurants dataframe
    rest_index = filtered_restaurants[filtered_restaurants['rest_name'] == rest_name].index[0]

    # Get the similarity scores for all restaurants
    rest_scores = combined_similarity[:, rest_index]

    # Sort the similarity scores
    sorted_indices = rest_scores.argsort()[::-1]

    # Get the top 15 recommended restaurants (excluding the target restaurant itself)
    top_recommendations = filtered_restaurants.iloc[sorted_indices]
    top_recommendations = top_recommendations[top_recommendations.index != rest_index][:15]

    recommended_rest = top_recommendations[['rest_name', 'address', 'rest_avg_stars', 'total_reviews', 'categories', 'review_text']].sort_values(by='rest_avg_stars', ascending=False)

    print('TOP %s RESTAURANTS LIKE %s WITH SIMILAR CATEGORIES AND REVIEWS: ' % (str(len(recommended_rest)), rest_name))

    return recommended_rest

# Example usage
rest_name = 'LA LA Bakeshop'
rating_tolerance = 4.0

recommendations = restaurant_recommendations(rest_name, rating_tolerance)
recommendations

TOP 15 RESTAURANTS LIKE LA LA Bakeshop WITH SIMILAR CATEGORIES AND REVIEWS: 


,rest_name,address,rest_avg_stars,total_reviews,categories,review_text
56,Grandma Loves You,"1084 Yonge Street, Toronto, ON M4W 2L4, Canada",5.0,75,sandwiches bakeries,around area last saturday bought spicy ham che...
93,Adobar,"200 Dundas St E, Toronto, ON M5A 1Z4, Canada",5.0,5,filipino comfort food,5 stars amazing 22 layer olive oil tsoko mouss...
155,Pho Madame Saigon,"2038 Yonge Street, Toronto, ON M4S 1Z9, Canada",4.5,7,vietnamese,absolutely amazing viet flavours madame saigon...
4,Le Conciliabule,"1300 Gerrard Street E, Toronto, ON M4L 1Y7, Ca...",4.5,13,bakeries wine bars bistros,meant come quite time never chance open hours ...
234,Saigon Lotus,"6 St Andrew Street, Toronto, ON M5T 1K6, Canada",4.5,215,vietnamese vegan vegetarian,back got goldenyellow curry vermicelli 17 larg...
232,Aloette,"163 Spadina Avenue, 1st Floor, Toronto, ON M5V...",4.5,292,french bistros diners,aloette came time time looking must try restau...
210,Yeah Yeah Thai,"382 Spadina Avenue, Toronto, ON M5T 2G5, Canada",4.5,17,thai hong kong style cafe,absolutely loved place must try gem chinatown ...
99,Nam Wan,"713 Gerrard St E, Toronto, ON J4Y 0K7, Canada",4.5,13,cafes thai,came place check thai tea cookies thai tea blo...
157,Flaming Stove,"21 Davisville Avenue, Toronto, ON M4S 1G3, Canada",4.5,67,middle eastern diners breakfast brunch,amazing shawarma wrap must get amba sauce ive ...
100,Que Ling Restaurant,"248 Boulton Avenue, Toronto, ON M4M 2K1, Canada",4.0,93,vietnamese,small family run pho place always takes us hap...
